**# Data Gathering**
only use data from last season to predict next season, as earlier seasons may give us bad estimates as quality of teams may have changed.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

loc = 'Dataset//'

data1 = pd.read_csv(loc + 'E0.csv')

df = [data1]


for ind, data in enumerate(df):
    data['Season'] = "{}/{}".format(18 - ind, 19 - ind)

col_list = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS',
            'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'Season']

all_data = pd.concat(df, sort='False')
all_data = all_data[col_list]

def result(HG, AG):
    if HG > AG:
        return -1
    elif HG == AG:
        return 0
    elif HG < AG:
        return 1

all_data['Result'] = all_data.apply(lambda row: result(row.FTHG, row.FTAG), axis = 1) 
print(all_data.shape)
all_data

(380, 18)


/var/folders/qk/cl2c3dyd16908blk55j0pfhr0000gn/T/ipykernel_4147/967873419.py:27: FutureWarning: Passing non boolean values for sort is deprecated and will error in a future version!
  all_data = pd.concat(df, sort='False')


,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,Season,Result
0,Crystal Palace,Arsenal,0,2,A,0,1,A,10,10,2,2,16,11,3,5,18/19,1
1,Fulham,Liverpool,2,2,D,1,0,H,9,11,3,4,7,9,4,4,18/19,0
2,Bournemouth,Aston Villa,2,0,H,1,0,H,7,15,3,2,18,16,5,5,18/19,-1
3,Leeds,Wolves,2,1,H,1,1,D,12,15,4,6,13,9,6,4,18/19,-1
4,Newcastle,Nott'm Forest,2,0,H,0,0,D,23,5,10,0,9,14,11,1,18/19,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Everton,Bournemouth,1,0,H,0,0,D,13,7,6,2,11,12,9,3,18/19,-1
376,Leeds,Tottenham,1,4,A,0,1,A,19,11,2,7,7,5,12,3,18/19,1
377,Leicester,West Ham,2,1,H,1,0,H,13,16,4,3,8,10,3,5,18/19,-1
378,Man United,Fulham,2,1,H,1,1,D,21,10,8,3,14,10,5,4,18/19,-1


**# Data Wrangling**

In [2]:
home_data = all_data.groupby('HomeTeam').sum()
away_data = all_data.groupby('AwayTeam').sum()
home_data['HAS'] = home_data['FTHG']/19 #home avg scores
home_data['HAC'] = home_data['FTAG']/19 # Home Average Conceded
away_data['AAS'] = away_data['FTAG']/19 # Away average Scored
away_data['AAC'] = away_data['FTHG']/19 # Away Average Conceded
all_data_backup = all_data
team_data = pd.DataFrame()
team_data['Team'] = home_data.index

team_data = team_data.merge(home_data['HAS'], left_on='Team', right_index=True)
team_data = team_data.merge(home_data['HAC'], left_on='Team', right_index=True)

team_data = team_data.merge(away_data['AAS'], left_on='Team', right_index=True)
team_data = team_data.merge(away_data['AAC'], left_on='Team', right_index=True)

team_data.set_index("Team", inplace=True)
last_season_table= pd.DataFrame(index=team_data.index)
team_data.to_csv("team_data.csv")
scaler = MinMaxScaler()
team_data = pd.DataFrame(scaler.fit_transform(team_data), columns=team_data.columns, index=team_data.index)
team_data





/var/folders/qk/cl2c3dyd16908blk55j0pfhr0000gn/T/ipykernel_4147/3564506129.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  home_data = all_data.groupby('HomeTeam').sum()
/var/folders/qk/cl2c3dyd16908blk55j0pfhr0000gn/T/ipykernel_4147/3564506129.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  away_data = all_data.groupby('AwayTeam').sum()


,HAS,HAC,AAS,AAC
Team,,,,
Arsenal,0.840909,0.555556,1.000000,0.071429
Aston Villa,0.386364,0.407407,0.291667,0.321429
Bournemouth,0.090909,0.666667,0.250000,0.964286
Brentford,0.431818,0.296296,0.500000,0.428571
Brighton,0.477273,0.407407,1.000000,0.571429
Chelsea,0.090909,0.333333,0.291667,0.428571
Crystal Palace,0.113636,0.481481,0.333333,0.357143
Everton,0.000000,0.629630,0.291667,0.500000
Fulham,0.340909,0.703704,0.541667,0.285714


In [3]:
col_list2 = ['HomeTeam', 'AwayTeam']
data_prep = all_data[col_list2]

data_prep = data_prep.merge(team_data['HAS'], left_on='HomeTeam', right_index=True, how='left')
data_prep = data_prep.merge(team_data['HAC'], left_on='HomeTeam', right_index=True, how='left') 
data_prep = data_prep.merge(team_data['AAS'], left_on='AwayTeam', right_index=True, how='left') 
data_prep = data_prep.merge(team_data['AAC'], left_on='AwayTeam', right_index=True, how='left') 

data_prep['HAtS'] = (data_prep['HAS'] * data_prep['AAC']); # Home Attack Strength
data_prep['AAtS'] = (data_prep['HAC'] * data_prep['AAS']); # Away Attack Strength

print(data_prep.shape)

data_prep = data_prep.merge(all_data['Result'], left_index=True, right_index=True)

fixture_list = data_prep

fixture_list.to_csv("fixture_list.csv")


data_prep

(380, 8)


,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS,Result
0,Crystal Palace,Arsenal,0.113636,0.481481,1.000000,0.071429,0.008117,0.481481,1
1,Fulham,Liverpool,0.340909,0.703704,0.750000,0.500000,0.170455,0.527778,0
2,Bournemouth,Aston Villa,0.090909,0.666667,0.291667,0.321429,0.029221,0.194444,-1
3,Leeds,Wolves,0.227273,1.000000,0.041667,0.785714,0.178571,0.041667,-1
4,Newcastle,Nott'm Forest,0.454545,0.148148,0.000000,1.000000,0.454545,0.000000,-1
...,...,...,...,...,...,...,...,...,...
375,Everton,Bournemouth,0.000000,0.629630,0.250000,0.964286,0.000000,0.157407,-1
376,Leeds,Tottenham,0.227273,1.000000,0.916667,0.785714,0.178571,0.916667,1
377,Leicester,West Ham,0.159091,0.629630,0.208333,0.535714,0.085227,0.131173,-1
378,Man United,Fulham,0.454545,0.000000,0.541667,0.285714,0.129870,0.000000,-1


In [4]:
rm_col_list = ['HomeTeam','AwayTeam']
col_list = data_prep.columns.values.tolist()
col_list = [x for x in col_list if x not in rm_col_list]

to_use = data_prep[col_list]
X = [x for x in col_list if x != 'Result']
Y = [x for x in col_list if x == 'Result']

to_use.to_csv("to_use.csv")

to_use

,HAS,HAC,AAS,AAC,HAtS,AAtS,Result
0,0.113636,0.481481,1.000000,0.071429,0.008117,0.481481,1
1,0.340909,0.703704,0.750000,0.500000,0.170455,0.527778,0
2,0.090909,0.666667,0.291667,0.321429,0.029221,0.194444,-1
3,0.227273,1.000000,0.041667,0.785714,0.178571,0.041667,-1
4,0.454545,0.148148,0.000000,1.000000,0.454545,0.000000,-1
...,...,...,...,...,...,...,...
375,0.000000,0.629630,0.250000,0.964286,0.000000,0.157407,-1
376,0.227273,1.000000,0.916667,0.785714,0.178571,0.916667,1
377,0.159091,0.629630,0.208333,0.535714,0.085227,0.131173,-1
378,0.454545,0.000000,0.541667,0.285714,0.129870,0.000000,-1
